In [2]:
import os
from jaf.core.llm.openai import AzureGPTLLM
from dotenv import load_dotenv
import json
import re
import subprocess

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
load_dotenv()

False

In [ ]:
AZURE_GPT_DEPLOYMENT_NAME=os.getenv("AZURE_GPT_DEPLOYMENT_NAME")
AZURE_OAI_BASE_URL=os.getenv("AZURE_OAI_BASE_URL")
AZURE_OAI_API_VERSION= os.getenv("AZURE_OAI_API_VERSION")
AZURE_OAI_API_KEY= os.getenv("AZURE_OAI_API_KEY")
azure_llm = AzureGPTLLM(
    deployment_name=AZURE_GPT_DEPLOYMENT_NAME,
    api_base=AZURE_OAI_BASE_URL,
    api_version=AZURE_OAI_API_VERSION,
    api_key=AZURE_OAI_API_KEY
)

OpenAIError: Missing credentials. Please pass one of `api_key`, `azure_ad_token`, `azure_ad_token_provider`, or the `AZURE_OPENAI_API_KEY` or `AZURE_OPENAI_AD_TOKEN` environment variables.

In [18]:
cmd = [
    "node", "../cada.js",
    "ssh://git@ssh.bitbucket.juspay.net/jbiz/rescript-euler-dashboard.git",
    "../rescript-euler-dashboard/",
    "3ba4fd29ca78f655aef39da221ea2925959751cd",
    "91187b10db4b7254c2a6759be9e5aab363f01dd5",
    "../rescript-euler-dashboard"
]

subprocess.run(cmd, check=True)

/Users/suryansh.s/cada-rescript/node_modules/tree-sitter/index.js:837
  for (let id = 0, n = nodeTypeNamesById.length; id < n; id++) {
                                         ^

TypeError: Cannot read properties of undefined (reading 'length')
    at initializeLanguageNodeClasses (/Users/suryansh.s/cada-rescript/node_modules/tree-sitter/index.js:837:42)
    at Parser.setLanguage (/Users/suryansh.s/cada-rescript/node_modules/tree-sitter/index.js:342:5)
    at Object.<anonymous> (/Users/suryansh.s/cada-rescript/cada.js:14:8)
    at Module._compile (node:internal/modules/cjs/loader:1734:14)
    at Object..js (node:internal/modules/cjs/loader:1899:10)
    at Module.load (node:internal/modules/cjs/loader:1469:32)
    at Function._load (node:internal/modules/cjs/loader:1286:12)
    at TracingChannel.traceSync (node:diagnostics_channel:322:14)
    at wrapModuleLoad (node:internal/modules/cjs/loader:235:24)
    at Function.executeUserEntryPoint [as runMain] (node:internal/modules/run_main:151

CalledProcessError: Command '['node', '../cada.js', 'ssh://git@ssh.bitbucket.juspay.net/jbiz/rescript-euler-dashboard.git', '../rescript-euler-dashboard/', '3ba4fd29ca78f655aef39da221ea2925959751cd', '91187b10db4b7254c2a6759be9e5aab363f01dd5', '../rescript-euler-dashboard']' returned non-zero exit status 1.

In [9]:
system_message = """You are a senior QA engineer specializing in frontend testing. Your task is to generate SPECIFIC, ACTIONABLE UI testing instructions based on code changes.

CRITICAL REQUIREMENTS:
- Be SPECIFIC about what to test, where to find it, and what to look for
- Provide DETAILED steps that any tester can follow without guessing
- Focus on REAL user scenarios and business workflows
- Include SPECIFIC UI elements, buttons, forms, pages to test
- Mention EXACT expected behaviors and visual changes
- Avoid generic phrases like "test the component" or "verify functionality"

When analyzing function/type changes, infer likely UI impact:
- New functions → new UI features/buttons/workflows to test
- Modified functions → changed behavior in existing UI elements
- Deleted functions → removed features that should no longer appear
- Type changes → data structure changes affecting forms, displays, validation

You MUST output a JSON object with exactly two keys:

1. "instructionPoints" — array of SPECIFIC test steps (8-15 words each)
2. "testInstructions" — detailed paragraph with concrete testing scenarios

GOOD Example:
{
  "instructionPoints": [
    "Open user profile page and locate new 'Export Data' button",
    "Click export button and verify download modal appears with format options",
    "Select CSV format and confirm file downloads with correct data",
    "Test export with empty profile and verify appropriate empty state message"
  ],
  "testInstructions": "Navigate to the user profile page (usually at /profile or /settings). Look for a new 'Export Data' button in the account settings section. Click this button and verify a modal dialog appears with export format options (CSV, JSON, PDF). Select CSV format and click 'Download' - confirm a file downloads with the user's profile data. Test edge case: create a test user with minimal profile data and verify the export shows appropriate empty state messages rather than blank fields."
}

BAD Example (too generic):
{
  "instructionPoints": [
    "Test the new functionality",
    "Verify the component works correctly",
    "Check for errors"
  ],
  "testInstructions": "Test the component to make sure it works properly and displays correctly."
}

Make sure the JSON is valid (no trailing commas), and do not wrap it in markdown fences. ALSO DO NOT MAKE ANY ASSUMPTION, WE WANT UNIT TEST CASES, NO ASSUMPTION FOR TRAVERSIG AND ALL"""

In [10]:
with open("detailed_changes.json", "r") as f:
    raw_changes = json.load(f)

In [11]:
test_plan = []

for change in raw_changes:
    module = change.get("moduleName", "Unknown")

    summaries = []
    summaries += [f"Added function `{name}` (likely affects UI component behavior)" for name, _ in change.get("addedFunctions", [])]
    summaries += [f"Modified function `{name}` (may change component interaction)" for name, _, _ in change.get("modifiedFunctions", [])]
    summaries += [f"Deleted function `{name}` (component functionality removed)" for name, _ in change.get("deletedFunctions", [])]
    summaries += [f"Added type `{name}` (new data structure for UI state)" for name, _ in change.get("addedTypes", [])]
    summaries += [f"Modified type `{name}` (UI data structure changed)" for name, _, _ in change.get("modifiedTypes", [])]
    summaries += [f"Deleted type `{name}` (UI data structure removed)" for name, _ in change.get("deletedTypes", [])]
    summaries += [f"Added external `{name}` (new external API/library integration)" for name, _ in change.get("addedExternals", [])]
    summaries += [f"Modified external `{name}` (external integration changed)" for name, _, _ in change.get("modifiedExternals", [])]
    summaries += [f"Deleted external `{name}` (external integration removed)" for name, _ in change.get("deletedExternals", [])]

    if not summaries:
        continue

    user_message = (
        f"Module **{module}** has the following changes:\n"
        + "\n".join(f"- {s}" for s in summaries)
        + "\n\n"
        "Based on these specific code changes, generate DETAILED UI testing instructions that include:\n\n"
        "MUST INCLUDE:\n"
        "- Specific page/URL paths where changes are likely visible\n"
        "- Exact UI elements (buttons, forms, menus, sections) to test\n"
        "- Step-by-step user actions with expected outcomes\n"
        "- Specific data to input and expected validation/error messages\n"
        "- Visual elements to verify (text, colors, layouts, animations)\n"
        "- Browser/device specific testing if relevant\n\n"
        "INFER FROM CHANGES:\n"
        "- If functions added → new features/buttons appeared\n"
        "- If functions modified → existing behavior changed  \n"
        "- If functions deleted → features removed/hidden\n"
        "- If types changed → form fields, validation, data display affected\n\n"
        "Provide actionable instructions a QA tester can follow immediately without needing to understand the code."
    )

    resp = azure_llm.call_llm(
        system_message=system_message,
        user_message=user_message,
        stream=False
    )
    instructions_raw = resp.choices[0].message.content.strip()
    instructions_raw = resp.choices[0].message.content.strip()

    parsed_instructions = None
    try:
        parsed = json.loads(instructions_raw)
        if isinstance(parsed, dict) and "testInstructions" in parsed:
            parsed_instructions = parsed
    except json.JSONDecodeError:
        try:
            inner = json.loads(json.loads(instructions_raw))
            if isinstance(inner, dict) and "testInstructions" in inner:
                parsed_instructions = inner
        except Exception:
            pass

    if parsed_instructions:
        test_instructions = parsed_instructions.get("testInstructions", "")
        instruction_points = parsed_instructions.get("instructionPoints", [])
    else:
        test_instructions = instructions_raw
        instruction_points = (
            re.findall(r'^\s*\d+\.\s*(.+)$', instructions_raw, flags=re.MULTILINE) or
            re.findall(r'^\s*####\s*\d+\.\s*(.+)$', instructions_raw, flags=re.MULTILINE)
        )

    entry = {
        "moduleName": module,
        "changes": {
            "addedFunctions":       [name for name, _ in change.get("addedFunctions", [])],
            "modifiedFunctions":    [name for name, _, _ in change.get("modifiedFunctions", [])],
            "deletedFunctions":     [name for name, _ in change.get("deletedFunctions", [])],
            "addedTypes":           [name for name, _ in change.get("addedTypes", [])],
            "modifiedTypes":        [name for name, _, _ in change.get("modifiedTypes", [])],
            "deletedTypes":         [name for name, _ in change.get("deletedTypes", [])],
            "addedExternals":       [name for name, _ in change.get("addedExternals", [])],
            "modifiedExternals":    [name for name, _, _ in change.get("modifiedExternals", [])],
            "deletedExternals":     [name for name, _ in change.get("deletedExternals", [])],
        },
        "testInstructions": test_instructions,
        "instructionPoints": instruction_points
    }

    test_plan.append(entry)

with open("test_plan.json", "w") as out:
    json.dump(test_plan, out, indent=2)

print(f"Generated test_plan.json with {len(test_plan)} entries focused on UI testing.")

Generated test_plan.json with 2 entries focused on UI testing.


In [12]:
result = json.loads(resp.choices[0].message.content)
instruction_points = result["instructionPoints"]
print(instruction_points)

['Navigate to the /login page', 'Locate the UPI login section on the page', 'Enter a valid UPI ID in the input field', "Click the 'Next' button to proceed", 'Check for error message with invalid UPI ID entry', "Verify loading animation appears after clicking 'Next'", 'Confirm redirection to Two-Factor Authentication page', 'Test form responsiveness on mobile and desktop browsers', 'Inspect text alignment and button visibility in different screen sizes']


In [13]:
full_instructions = result["testInstructions"]
print((full_instructions))

Navigate to the login page at the URL path /login or based on the application's starting point that leads to this login process. Within this page, locate the new UPI login section, which should be prominent possibly due to its recent modifications. Inside this section, find the input field specifically for UPI ID entry. Begin testing by entering a valid UPI ID and press the 'Next' button to move forward in the login process. Observe that a loading animation should appear briefly to signify processing, after which you should expect to be redirected to the Two-Factor Authentication page. This is crucial as it tests the new workflow adjustments declared in the 'make' function update. In contrast, deliberately enter an invalid UPI ID (e.g., a random string not formatted as a UPI ID) and ensure the UI displays a meaningful error message alerting the user to enter a correct UPI ID. For responsiveness, adjust the browser window size across typical screen dimensions, such as mobile or desktop,